<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/YouTube_to_MP3maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title YouTubeプレイリストの音声を順番通りにダウンロードして結合するスクリプト
from google.colab import files
from datetime import datetime
import os
import shutil
import subprocess

# プレイリストURL
playlist_url = "" #@param {type:"string"}

# 必要なパッケージのインストール
!pip install -q yt-dlp ffmpeg-python

# 現在の日時を取得
now = datetime.now()
formatted_now = now.strftime('%Y%m%d%H%M%S')

# ディレクトリ設定
base_dir = '/content/playlist_download'
output_dir = '/content'
os.makedirs(base_dir, exist_ok=True)
os.chdir(base_dir)

# 出力ファイル名
output_file_name = f'playlist_output_{formatted_now}.mp3'
full_output_path = os.path.join(output_dir, output_file_name)

try:
    # MP3形式でプレイリストの音声を順番通りにダウンロード
    download_command = [
        'yt-dlp',
        '-x',  # Extract audio
        '--audio-format', 'mp3',  # Convert to MP3
        '--yes-playlist',  # Download entire playlist
        '--playlist-items', '1-',  # プレイリストの全アイテムをダウンロード
        '--output', '%(playlist_index)03d.%(ext)s',  # 001, 002, 003の形式でファイル名を作成
        playlist_url
    ]

    # コマンド実行と出力の表示
    result = subprocess.run(download_command, capture_output=True, text=True)
    print("Download command output:", result.stdout)
    print("Download command errors:", result.stderr)

    # MP3ファイルのリストを順番通りに取得
    mp3_files = sorted([f for f in os.listdir() if f.endswith('.mp3')])

    if not mp3_files:
        raise ValueError("No MP3 files were downloaded")

    print(f"Found {len(mp3_files)} MP3 files: {mp3_files}")

    # FFmpegを使用してファイル結合
    with open('file_list.txt', 'w') as f:
        for mp3_file in mp3_files:
            f.write(f"file '{mp3_file}'\n")

    # 結合コマンド
    merge_command = [
        'ffmpeg',
        '-f', 'concat',
        '-safe', '0',
        '-i', 'file_list.txt',
        '-c', 'copy',
        full_output_path
    ]

    # 結合コマンド実行と出力の表示
    merge_result = subprocess.run(merge_command, capture_output=True, text=True)
    print("Merge command output:", merge_result.stdout)
    print("Merge command errors:", merge_result.stderr)

    # ファイルの存在を確認
    if not os.path.exists(full_output_path):
        raise FileNotFoundError(f"Output file not created: {full_output_path}")

    # ファイルをダウンロード
    print(f"Attempting to download: {full_output_path}")
    files.download(full_output_path)

    print(f"Successfully downloaded and merged {len(mp3_files)} MP3 files.")

except Exception as e:
    print(f"An error occurred: {e}")
    # 追加のデバッグ情報
    print("Current working directory:", os.getcwd())
    print("Files in current directory:", os.listdir())

finally:
    # クリーンアップ: playlist_downloadディレクトリの中身を削除
    os.chdir('/content')
    try:
        shutil.rmtree(base_dir)
        print(f"Deleted temporary directory: {base_dir}")
    except Exception as e:
        print(f"Error deleting temporary directory: {e}")